In [1]:
## Math/ML imports
import tensorflow as tf
from tensorflow.keras import backend as K
import numpy as np
#import math

## Metrics
#from sklearn.metrics import f1_score, precision_score, recall_score
#from sklearn.metrics import confusion_matrix

## Garbage collector
import gc

## Helpers
from png_helper import load_data

## Time
#import time
#from datetime import timedelta

## Graphical
#%matplotlib inline
#import matplotlib.pyplot as plt
#from sklearn.metrics import confusion_matrix


In [2]:
# Load data
(x_train, y_train), x_test = load_data()

# Map from 0-255 to 0-1
x_train = x_train / 255.0
y_train = y_train / 255.0
x_test = x_test / 255.0

# Turn the train set into binary (condition: >=0.5)
y_train = tf.keras.utils.to_categorical(y_train * 2) # two categorical collumns
y_train = y_train[:,:,:,1:2] # keep only second collumn. (road=1, background=0)

In [3]:
x_train.shape, x_test.shape, y_train.shape

((100, 400, 400, 3), (50, 608, 608, 3), (100, 400, 400, 1))

In [4]:
y_train[0,10,185:195,0] # road=1, background=0

array([0., 0., 0., 0., 0., 0., 1., 1., 1., 1.], dtype=float32)

In [5]:
#from showim import showim
#showim(x_train[0], x_train[1], x_train[2], y_train[0], y_train[1], y_train[2])

In [6]:
x_train[0][10][200]

array([0.24705882, 0.23137255, 0.21568627])

In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(128, (10, 10), strides=2,
                         activation=tf.nn.relu), # 399 -> 195
  tf.keras.layers.Conv2D(256, (6, 6), strides=2,
                         activation=tf.nn.relu), # 195 -> 95
  tf.keras.layers.Conv2D(256, (6, 6),
                         activation=tf.nn.relu), # 95 -> 90
  tf.keras.layers.Conv2D(1024, (7, 7), strides=2,
                         activation=tf.nn.relu), # 90 -> 42
  tf.keras.layers.Conv2D(1024, (5, 5),
                         activation=tf.nn.relu), # 42 -> 38
  tf.keras.layers.Conv2DTranspose(1, (20, 20), strides=10, # 38 -> 399
                         activation=None)
])

In [8]:
def mcor(y_true, y_pred):
    #matthews_correlation
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos
    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos
    
    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)
    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)
    
    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    
    return numerator / (denominator + K.epsilon())

def bcor(y_true, y_pred):
    pp = K.mean(K.round(K.clip(y_pred, 0, 1)))
    pn = 1 - pp
    pos = K.mean(K.round(K.clip(y_true, 0, 1)))
    neg = 1 - pos
    
    tp = K.mean(K.round(K.clip(y_true * y_pred, 0, 1)))
    fp = pp - tp
    
    fn = pos - tp
    tn = pn - fn
    
    return (tp - (pp*pos)) / (pos - (pos*pos))

def precision(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    pp = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = tp / (pp + K.epsilon())
    return precision

def recall(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    pos = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = tp / (pos + K.epsilon())
    return recall

def f1(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    pp = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = tp / (pp + K.epsilon())
    
    pos = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = tp / (pos + K.epsilon())
    
    return 2*((precision * recall) / (precision + recall + K.epsilon()))

In [9]:
gc.collect()

0

In [10]:
# Compile model
optimizer = tf.keras.optimizers.Adam(lr=0.10, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy', f1, precision, recall, mcor, bcor])

In [11]:
# Train
model.fit(x_train, y_train, batch_size=20, epochs=100)

Epoch 1/100
100/100 [==============================] - 31s 309ms/step - loss: 210.2314 - acc: 0.3323 - f1: 0.2716 - precision: 0.1640 - recall: 0.7920 - mcor: -1.0892e-04 - bcor: -2.9205e-05
Epoch 2/100
100/100 [==============================] - 12s 122ms/step - loss: 253.0266 - acc: 0.2065 - f1: 0.3332 - precision: 0.2005 - recall: 0.9900 - mcor: -1.3364e-04 - bcor: -3.3277e-05
Epoch 3/100
100/100 [==============================] - 12s 121ms/step - loss: 253.0266 - acc: 0.2065 - f1: 0.3334 - precision: 0.2005 - recall: 0.9900 - mcor: -1.3310e-04 - bcor: -3.3031e-05
Epoch 4/100
100/100 [==============================] - 12s 121ms/step - loss: 253.0266 - acc: 0.2065 - f1: 0.3331 - precision: 0.2005 - recall: 0.9900 - mcor: -1.3368e-04 - bcor: -3.3368e-05
Epoch 5/100
100/100 [==============================] - 12s 121ms/step - loss: 253.0266 - acc: 0.2065 - f1: 0.3332 - precision: 0.2005 - recall: 0.9900 - mcor: -1.3623e-04 - bcor: -3.4591e-05
Epoch 6/100
100/100 [========================

KeyboardInterrupt: 

In [ ]:
# Evaluate
model.evaluate(x_train, y_train)

# Some blank lines
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>